In [ ]:
import torch
import torchvision
import torch.nn as nn 
import torch.nn.init
import numpy as np
import torchvision.transforms as transforms 
import matplotlib.pyplot as plt

#디버깅 모듈 설치
import pdb




In [ ]:
#우리가 사용할 컴퓨터를 check하는 부분, cpu/gpu 지원받을지..
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
# 하이퍼파라미터 설정

input_size = 3072 # mnist는 32*32*3로 구성
hidden_size = 500 # 은닉층의 unit 수
num_classes = 10 # 카테고리 개수
num_epochs = 5
batch_size = 100
learning_rate = 0.001

# dataset 로딩 - 2개의 과정을 거친다
# train_dataset, test_dataset 데이터 로딩 1단계
train_dataset = torchvision.datasets.CIFAR10(root='../../data',
                                           train = True, #traindataset만 저장
                                           transform = transforms.ToTensor(),
                                           download = True) #traindataset을 스케일링하여 다운로드하겠다.
test_dataset = torchvision.datasets.CIFAR10(root='../../data',
                                           train = False, #test_dataset만 저장, test옵션은 따로 없음.
                                           transform = transforms.ToTensor())

# 데이터 로딩 2단계.. BatchSize 이용
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size = batch_size,
                                           shuffle =True)

# 100개씩 쪼개서 저장..
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size = batch_size,
                                           shuffle =False) #테스트는 학습이아니므로 shuffle 필요 없다.

In [ ]:
class ConvNet(nn.Module):
  def __init__(self, num_classes=10):
    super(ConvNet, self).__init__()

        # L1 ImgIn shape=(?, 32, 32, 3)
        #    Conv     -> (?, 32, 32, 32)
        #    Pool     -> (?, 16, 16, 32)
    self.layer1 =nn.Sequential(
        nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(32),#
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2))

        
        
        # L2 ImgIn shape=(?, 16, 16, 32)
        #    Conv     -> (?, 16, 16, 64)
        #    Pool     -> (?, 8, 8, 64)   
    self.layer2 = nn.Sequential(
        nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(64), #
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2))
    

        # L3 ImgIn shape=(?, 8, 8, 64)
        #    Conv     -> (?, 8, 8, 128)
        #    Pool     -> (?, 4, 4, 128)   
    self.layer3 = nn.Sequential(
        nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(128), #
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)


    #     # L3 ImgIn shape=(?, 8, 8, 64)
    #     #    Conv     -> (?, 8, 8, 128)
    #     #    Pool     -> (?, 4, 4, 128)   
    # self.layer4 = nn.Sequential(
    #     nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
    #     nn.BatchNorm2d(256), #
    #     nn.ReLU(),
    #     nn.MaxPool2d(kernel_size=2, stride=2)


    self.fc1 = torch.nn.Linear(4*4*128, 625, bias=True)
    torch.nn.init.xavier_uniform_(self.fc1.weight)
    self.layer3 = torch.nn.Sequential(
        self.fc1,
        torch.nn.ReLU(),
        torch.nn.Dropout(p=1 - self.keep_prob))
    # L5 Final FC 625 inputs -> 10 outputs
    self.fc2 = torch.nn.Linear(625, num_classes,  bias=True)
    torch.nn.init.xavier_uniform_(self.fc2.weight)

  def forward(self, x):

    # pdb.set_trace() # 2.디버깅
        # n : 다음 라인 가르킨다.
        # x.size, out.size  등 크기 확인 필요

        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)   # Flatten them for FC
        out = self.layer3(out)
        out = self.fc2(out)
        return out



In [ ]:
model = ConvNet(num_classes).to(device)

In [ ]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [ ]:
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # pdb.set_trace() # 1. 디버깅.. 주석으로 막아두고 후에 사용.. outputs에서 호출되며 실행 
         
        # l : pdb로 어느부분을 디버깅 하려 하는지 알 수 있음.. 호출되는 함수인 forward가 나타남.
        # images.size(), images.shape : 이미지 사이즈 알아보기([batch_size, channel, height, width])
        # labels.size() : 라벨의 사이즈 알 수 있다.
        # labels : 라벨을 전부 뽑는다.
        # quit, q : 빠져나온다. 빠져나오면서 error 나오지만 신경 안써도됨.
        

        

        outputs = model(images) # =model.forward(images)...모델에 입력값을 넣어 예측값을 반환. 이때 forward함수 호출.
        
        loss = loss_function

        # pdb.set_trace() #3.디버깅
            # loss값 확인 가능. loss.item()..
        n(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if(i+1) % 100 ==0:
            print('Epoch[{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, total_step, loss.item()))